mapper：   
$\hat{\beta}=(X^T X)^{-1}X^T Y$:    
step1:$X^T Y$ 视为X的第一行每个元素与Y的第一个元素对应相乘（同属输入数据的一行），得到p+1个数（含常数1与Y相乘），每读入一行就累加在上一次的结果上，获得新的p+1个数。    
step2:$X^T X$ 视为p+1个$X^T$与一个列向量相乘得到的p+1个长度为p+1的列向量，即一个方阵，每读入一行对整个方阵进行累加更新。    

In [4]:
#! /usr/bin/env python
# -*- coding:UTF-8 -*-
import numpy as np
import sys

pplus1 = None # p+1
for line in sys.stdin:
    line = line.strip().split(",")
    line.insert(0,1) # 在每行开头插入一个常数1，便于求出截距项
    line = np.array(line,float)
    
    if pplus1 == None: # 读入第一行时初始化X转置*Y和X转置*X
        pplus1 = len(line) - 1
        XY = np.zeros(pplus1) # p+1维向量
        XTX = np.zeros(shape=(pplus1,pplus1)) # (p+1)*(p+1)的矩阵
    
    XY += line[:pplus1] * line[pplus1] # 一行中X部分元素分别乘以该行的Y元素，累加到上一行的结果上
    for i in range(pplus1):
        XTX[i] += line[:pplus1] * line[i]
    
out = XY.tolist() + XTX.reshape(1,pplus1**2)[0].tolist() # 将XTX拉成一行并与XY连接
print(",".join(str(i) for i in out),file=sys.stdout) # 标准化输出

203.48878742400044,13228.129652247759,18605.876607883263,30285.421426016746,41455.04222273555,50946.07877453317,58185.47128184575,71705.87875643143,81593.36487598107,10000.0,31.188572609999827,-43.97876628600033,-93.39180559099988,-2.0534895459999305,-27.24595332900002,36.24811815700011,-36.20574126300007,86.11490490699998,31.188572609999827,9876.63309427965,-55.62470243744511,-111.50355299373207,58.96455765714138,259.9086729149925,30.968317623095547,105.81063517806164,162.2884458280009,-43.97876628600033,-55.62470243744511,9682.303704300644,-34.759578450444565,-85.53105618075249,0.4684332982576983,-9.681483359138502,106.6829469416877,-116.2275292963012,-93.39180559099988,-111.50355299373207,-34.759578450444565,10179.169774967735,243.51243046416246,96.3341969503558,-19.49836658668663,-61.610310550712896,-122.78651978618073,-2.0534895459999305,58.96455765714138,-85.53105618075249,243.51243046416246,10265.126825459662,-52.80520694390389,10.105955702930629,56.86379307305451,-49.9780702035

reducer:   
将mapper的输出对应相加，得到最终的$(X^T X)$(拉直为向量)和$X^T Y$,总长度为$(p+1)^2+(p+1)$   
将上述结果还原为原来的维度，进一步计算得到$\hat \beta$

In [6]:
#! /usr/bin/env python
# -*- coding:UTF-8 -*-
import numpy as np
import sys
import math

length = None # XY和XTX结果的连接长度
for line in sys.stdin:
    line = line.strip().split(",")
    line = np.array(line,float)
    if length == None: # 初始化XY和XTX的连接向量
        length = len(line)
        XY_XTX = np.zeros(length)
    XY_XTX += line # 将每个mapper的输出对应加总

pplus1 = int((-1+math.sqrt(1+4.0*length))/2.0) # 解二次方程得到参数个数p+1
XY = np.matrix(XY_XTX[:pplus1]).T # 还原为列向量（矩阵）
XTX = np.matrix(XY_XTX[pplus1:]).reshape(pplus1,pplus1) # 还原为矩阵
beta_hat = (XTX.I*XY).reshape(1,pplus1).tolist()[0]
print(",".join(str(i) for i in beta_hat),file=sys.stdout) # 标准化输出

0.0031398312620881386,1.0036754336145082,1.9982438515015932,3.000480267722935,3.998170518095682,5.0019542805180315,6.000620444896159,6.996922418991534,8.003244254405267


验证：

In [8]:
f=open("test.csv")
datain=f.readlines()
f.close()
x=[]
y=[]
for line in datain: # 将mapper的输出对应相加
    fields = line.strip().split(",")
    x.append(fields[:-1])
    y.append(fields[-1])

In [10]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
print(reg.intercept_,reg.coef_)

0.0031398312620883433 [1.00367543 1.99824385 3.00048027 3.99817052 5.00195428 6.00062044
 6.99692242 8.00324425]


计算$\hat \sigma$与$\hat \beta$的标准差: 再进行一轮MapReduce，将前面求解得到的$\hat \beta$和$X^TX$设置为master的环境变量，通过streaming的-cmdenv参数分配给各个节点。   
mapper：利用每行观测的值计算$\hat y$，计算得到残差，并平方，输出该节点上处理的行数和该部分残差平方和。    
reducer: 将mapper的输出加总，得到总的观测数和总的残差平方和，利用$X^TX$的长度求出变量个数P，则可计算的得到$\hat \sigma$，进一步计算得到$\hat \beta$的标准差

In [13]:
#! /usr/bin/env python
# -*- coding:UTF-8 -*-
import sys
import os
import numpy as np

beta_hat=os.environ.get('beta_hat') # 获取环境变量
beta_hat=np.array(beta_hat.strip().split(","),float)
sumres = 0 # 初始化残差平方和
pplus1 = len(beta_hat) # 参数个数

for line in sys.stdin:
    line = line.strip().split(",")
    line.insert(0,1) # 在每行开头插入一个常数1
    line = np.array(line,float)
    sumres += (sum(line[:pplus1]*beta_hat)-line[pplus1])**2 # 该部分的残差平方和
    
out = [len(sys.stdin),sumres] #输出该部分的行数和残差平方和
    
print(",".join(str(i) for i in out),file=sys.stdout) # 标准化输出

10000,1591.8434191277745


In [24]:
#! /usr/bin/env python
# -*- coding:UTF-8 -*-
import sys
import numpy as np
import math

n = 0
sumresall = 0 
for line in sys.stdin:
    line = line.strip().split(",")
    line = np.array(line,float)
    n += line[0]
    sumresall += line[1]

XTX=os.environ.get('XTX')
XTX = np.array(XTX.strip().split(","),float)
pplus1 = int(math.sqrt(len(XTX)))
XTX = np.matrix(XTX).reshape(pplus1,pplus1)
sigma_hat = math.sqrt(sumresall/(n-pplus1))
stderr_beta = (sigma_hat*np.sqrt(XTX.I.diagonal())).tolist()[0]

beta_hat=os.environ.get('beta_hat')
beta_hat=np.array(beta_hat.strip().split(","),float)
print("beta_hat: "+",".join(str(i) for i in beta_hat),file=sys.stdout) # 标准化输出
print("sigma_hat: "+str(sigma_hat),file=sys.stdout) # 标准化输出
print("stderr_beta: "+",".join(str(i) for i in stderr_beta),file=sys.stdout) # 标准化输出

beta_hat: 0.0031398312620881386,1.0036754336145082,1.9982438515015932,3.000480267722935,3.998170518095682,5.0019542805180315,6.000620444896159,6.996922418991534,8.003244254405267
sigma_hat: 0.3991587865448609
stderr_beta: 0.003992026074224124,0.004018970146240751,0.004057343070182796,0.003958413839965314,0.003941208288765209,0.0039652322693902115,0.004069787458981472,0.003943138063489012,0.003975666688356989


bash脚本：

In [ ]:
#! /bin/bash
hadoop fs -rm -r yangle/lmout1

hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input yangle/test.csv \
-output yangle/lmout1 \
-file ./lmmapper.py \
-file ./lmreducer.py \
-mapper "python lmmapper.py" \
-reducer "python lmreducer.py" \
-numReduceTasks 1

hadoop fs -cat yangle/lmout1/part-*

export beta_hat=$(hadoop fs -cat yangle/lmout1/part-* | tail -n 1)
echo ${beta_hat}
export XTX=$(hadoop fs -cat yangle/lmout1/part-* | head -n 1)
echo ${XTX}

hadoop fs -rm -r yangle/lmout2
hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input yangle/test.csv \
-output yangle/lmout2 \
-file ./lmsigma_mapper.py  \
-file ./lmsigma_reducer.py \
-mapper "python lmsigma_mapper.py" \
-reducer "python lmsigma_reducer.py" \
-cmdenv "beta_hat=${beta_hat}" \
-cmdenv "XTX=${XTX}" \
-numReduceTasks 1

hadoop fs -cat yangle/lmout2/part-*